In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [9]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [11]:
block_size = 3
X, Y = [], []
for w in words[:5]:
    print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '---->', itos[ix])
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)




emma
... ----> e
..e ----> m
.em ----> m
emm ----> a
mma ----> .
olivia
... ----> o
..o ----> l
.ol ----> i
oli ----> v
liv ----> i
ivi ----> a
via ----> .
ava
... ----> a
..a ----> v
.av ----> a
ava ----> .
isabella
... ----> i
..i ----> s
.is ----> a
isa ----> b
sab ----> e
abe ----> l
bel ----> l
ell ----> a
lla ----> .
sophia
... ----> s
..s ----> o
.so ----> p
sop ----> h
oph ----> i
phi ----> a
hia ----> .


In [14]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [15]:
C = torch.randn((27, 2))
C

tensor([[-1.0701,  0.1680],
        [ 2.7994,  1.5347],
        [ 1.2908, -0.8884],
        [-0.2745,  1.0451],
        [ 0.1951,  1.3644],
        [ 0.3615, -0.6008],
        [ 0.4458,  0.6037],
        [-1.1416,  0.3112],
        [-0.2446, -0.9147],
        [-0.8479, -0.2398],
        [ 1.2491,  0.1542],
        [ 0.9352,  0.4083],
        [ 0.1389,  0.2081],
        [-0.6831, -1.5429],
        [-0.0319, -1.8108],
        [ 1.4659,  0.3422],
        [ 1.6660, -1.0304],
        [ 2.3522, -0.7028],
        [ 1.7209,  0.7499],
        [-1.8036, -0.8170],
        [ 0.0273,  0.5074],
        [ 1.2648, -1.3527],
        [ 0.6486, -0.4228],
        [-0.4516,  0.6498],
        [-0.6505,  0.9247],
        [ 0.6661, -0.7679],
        [-0.8281, -0.3330]])

In [33]:
emb = C[X]

In [34]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [30]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)

In [36]:
h.shape

torch.Size([32, 100])

In [35]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [37]:
logits = h @ W2 + b2

In [41]:
counts = logits.exp()

In [45]:
prob = counts / counts.sum(1, keepdim=True)

In [48]:
prob[0].sum().dtype

torch.float32

In [52]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(15.9199)